# Lectura 26: DataFrame - Manipulación y selección II

## `cast`

Esta función castea las columnas de un DataFrame a un tipo de datos en específico. Recibe como parámetro un diccionario con el nombre de la(s) columna(s) que se desean castear y el tipo de dato al cual se van a castear.

In [ ]:
import polars as pl
from datetime import date

df = pl.DataFrame(
    {
        "num": [1, 2, 3],
        "dec": [6.0, 7.0, 8.0],
        "date": [date(2024, 1, 2), date(2024, 3, 4), date(2023, 5, 6)],
        "date1": [date(2024, 5, 23), date(2024, 3, 14), date(2023, 5, 26)]
    }
)

df

In [ ]:
df.cast({'num': pl.Float32, 'dec': pl.UInt8})

Podemos castear todas las columnas de un tipo de datos específico a otro tipo de datos usando selectores.

In [ ]:
import polars.selectors as cs

df.cast({cs.date(): pl.Datetime})

## `clone`

Con esta función podremos crear una copia de un DataFrame. Esta es una operación poco costosa porque no copia los datos.

In [ ]:
vuelos = pl.read_parquet('./data/vuelos/', use_pyarrow=True)

vuelos_copy = vuelos.clone()

vuelos_copy

## `explode`

Realiza un explode del DataFrame a un formato más largo al realizar un explode de las columnas proporcionadas. Para visualizar como funciona explode vamos a crear un nuevo DataFrame.

In [ ]:
df_compacto = pl.DataFrame(
    {
        'letras': ['x', 'x', 'z', 'y'],
        'num': [[1], [2,3], [4,5], [6,7,8]]
    }
)

df_compacto

In [ ]:
df_explode = df_compacto.explode('num')

df_explode

## `hstack`

Esta función retorna un nuevo DataFrame creciendo horizontalmente un DataFrame existente al agregarle múltiples series. Recordemos como está constituido el DataFrame `df_compacto` y empleemos `hstack` para crecerlo horizontalmente. 

In [ ]:
df_compacto

In [ ]:
colores = pl.Series('colores', ['rojo', 'verde', 'azul', 'verde'])

decimal = pl.Series('decimal', [1.2, 3.5, 5.3, 9.0])

In [ ]:
df_extendido = df_compacto.hstack([colores, decimal])

df_extendido

## `vstack` y `extend`

### `vstack`

Esta función crece el DataFrame verticalmente apilándole un DataFrame. Para ver su funcionamineto vamos a utulizar el Dataframe `vuelos` y el DataFrame `vuelos_copy` que previamente hemos creado.

In [ ]:
vuelos.vstack(vuelos_copy)

Esta función devuelve un nuevo DataFrame a menos que se especifique el parámetro `in_place=True`.

In [ ]:
# Verificamos que el DataFrame vuelos no halla sido modificado

vuelos.shape

In [ ]:
vuelos.vstack(vuelos_copy, in_place=True)

In [ ]:
# Volvemos a verificar el DataFrame vuelos y veremos como ha sido modificado

vuelos.shape

### `extend`

Esta función amplía la memoria respaldada por el DataFrame al cual se le aplica con los valores del DataFrame que se extiende.

A diferencia de `vstak`, que agrega los fragmentos del DataFrame que se pasa como parámetro a los fragmentos del DataFrame, `extend` agrega los datos del DataFrame pasado como parámetro a las ubicaciones de memoria subyacentes y, por lo tanto, puede provocar una reasignación.


Prefiera `extend` sobre `vstack` cuando desee realizar una consulta después de un solo append. Por ejemplo, durante operaciones en línea en las que agrega n filas y vuelve a ejecutar una consulta.

Prefiera `vstack` sobre `extend` cuando desee agregar muchas veces antes de realizar una consulta. Por ejemplo, cuando lee varios archivos y desea almacenarlos en un único DataFrame. En el último caso, finalice la secuencia de operaciones vstack con un `rechunk`.

Este método modifica el DataFrame in-place. El DataFrame es devuelto solo por conveniencia.

Para mostrar el funcionaminto de `extend` vamos a leer algunas particiones del DataFrame de vuelos que se encuentaran dentro de la carpeta vuelos_particionado y vamos a unirlos con `extend`.

In [ ]:
vuelos_AA = pl.read_parquet('./data/vuelos_particionado/AIRLINE=AA/', use_pyarrow=True)

vuelos_AS = pl.read_parquet('./data/vuelos_particionado/AIRLINE=AS/', use_pyarrow=True)

vuelos_B6 = pl.read_parquet('./data/vuelos_particionado/AIRLINE=B6/', use_pyarrow=True)

In [ ]:
vuelos = vuelos_AA.extend(vuelos_AS).extend(vuelos_B6)

In [ ]:
vuelos

## `partition_by`

Esta función agrupa por las columnas proporcionadas y retorna los grupos como DataFrames separados en una lista.

Tomemos el DataFrame de `vuelos` que acabamos de crear y particionémoslo por la columna `MONTH`.

In [ ]:
vuelos_por_mes = vuelos.partition_by('MONTH')

vulelos_por_mes

In [ ]:
vuelos_por_mes[1]

En caso de que deseemos que retorne los DataFrame en un diccionario podemos utilizar el parámetro `as_dict=True`.

In [ ]:
vuelos_por_mes_dict = vuelos.partition_by('MONTH', as_dict=True)

vuelos_por_mes_dict

In [ ]:
vuelos_por_mes_dict.get(3)

## `rename`

Esta función permite renombrar las columnas del DataFrame. 

In [ ]:
vuelos.rename({'MONTH': 'mes', 'DAY': 'dia'})

## `with_columns`

Esta función permite agregar columnas al DataFrame. Si el nombre de la columna agregada coincide con un nombre de columna existente entonces se reemplazará la columna existente por la nueva columna.

In [ ]:
from polars import col

vuelos.with_columns((col('DAY_OF_WEEK') * 10).alias('day_of_week_10'))

Si no especificamos el nuevo nombre de columna se sobreescribirá la columna existente en el DataFrame.

In [ ]:
vuelos.with_columns((col('DAY_OF_WEEK') * 10))

También podemos agregar varias columnas en una sola ejecución. Para ello debemos proporcionar las nuevas columnas en una lista como se muestra a continuación.

In [ ]:
vuelos.with_columns(
    [
        (col('YEAR') + 1).alias('year_plus_1'),
        (col('AIR_TIME') / 60).alias('air_time_hrs'),
        col('TAIL_NUMBER').str.replace('N3','JO')
    ]
)

## `unique`

Esta función elimina las filas duplicadas del DataFrame. Si no se le proporciona ningún parámetro usará todas las columnas para identificar las filas duplicadas y eliminarlas. En caso de que se desee indicar por cual columna(s) se debe aplicar el borrado se deberán proporcionar el parámetro `subset=[col1, col2, ..., colN]`.

Para mostrar su funcionamiento creemos un nuevo DataFrame.

In [ ]:
df = pl.DataFrame(
    {
        'id': [1,2,3,1],
        'col_a': ['a', 'a', 'a', 'a'],
        'col_b': ['b', 'b', 'b', 'b']
    }
)

In [ ]:
df.unique()

Podemos mantener el orden del DataFrame original con el parámetro `maintain_order=True`. Esta operación es más costosa de calcular.

In [ ]:
df.unique(maintain_order=True)

Podemos indicarle la(s) columna(s) a considerar para identificar las filas duplicadas.

In [ ]:
df.unique(subset=['col_a', 'col_b'])